In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    import functools, traitlets, dataclasses, ipywidgets, IPython
    from pidgin import formatter, weave, shell

In [3]:
    class Manager(shell.Shell, IPython.core.formatters.BaseFormatter):
        container = ipywidgets.VBox()
        annotations = traitlets.Dict()
        widgets = traitlets.Dict()
        parent = traitlets.Instance(IPython.InteractiveShell)
        type_printers = traitlets.Dict(    dict((
            (tuple, ipywidgets.interaction.interactive.widget_from_abbrev), 
            (str, ipywidgets.interaction.interactive.widget_from_abbrev),
            (int, ipywidgets.interaction.interactive.widget_from_abbrev),
            (float, ipywidgets.interaction.interactive.widget_from_abbrev),
            (list, ipywidgets.interaction.interactive.widget_from_abbrev),
            (dict, ipywidgets.interaction.interactive.widget_from_abbrev),
            (set, ipywidgets.interaction.interactive.widget_from_abbrev),
        )))
        
        def run_cell(Manager, *args, **kwargs):
            app = Manager.parent
            __annotations__ = app.user_ns.get('__annotations__', {})
            for name, value in __annotations__.items():
                if type(value) != type(Manager.annotations.get(name, None)):
                    Manager.annotations[name] = value
                    Manager.widgets[name] = Manager.lookup(value)(value)
                    Manager.widgets[name].description = Manager.widgets[name].description or name
                    Manager.widgets[name].observe(functools.partial(Manager.observer, name), names='value')
                Manager.observer(name)            
                    
        def observer(Manager, key, change=None):
            Manager.user_ns[key] = Manager.widgets[key].value = Manager.user_ns.get(key, Manager.widgets[key].value)
            
        def _ipython_display_(Manager): 
            Manager.container.children = list(Manager.widgets.values())
            IPython.display.display(Manager.container)

In [4]:
    def load_ipython_extension(app):
        app.manager = Manager(user_module=app.user_module, parent=app, parents=[app])
        app.user_ns['app'] = app
        app._ipython_display_ = app.manager._ipython_display_
        app.events.register('post_run_cell', app.manager)

In [5]:
    def unload_ipython_extension(app):
        del app._ipython_display_
        pops = []
        for i, object in enumerate(app.events.callbacks['post_run_cell']):
            if isinstance(object, Manager): pops.append(i)
        for i in reversed(pops): app.events.callbacks['post_run_cell'].pop(i)
    if __name__ == '__main__': load_ipython_extension(get_ipython())